# Seldon-Core Component Demo

If you are reading this then you are about to take Seldon-Core, a model serving framework, for a test drive.

Seldon-Core has been packaged as a [combinator component](https://combinator.ml/components/introduction/), which makes it easy to spin up a combination of MLOps components to make a stack. This notebook is running within the cluster, next to the Seldon-Core installation.

The following demo is a very short introduction to show you how to connect to seldon-core. But I recommend that you follow the [official documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/github-readme.html) for a comprehensive guide.

## Prerequisites

You will primarily interact with Seldon-Core via the Kubernetes API. This means we need to download `kubectl`.

`kubectl` usage, however, requires permission. This notebook needs permission to perform actions on the Kubernetes API. This is acheived in the test drive codebase by connecting the seldon-core operator cluster role to the default service account.

:warning: Connecting pre-existing cluster roles to default service accounts is not a good idea! :warning:

In [1]:
!wget -q -O /tmp/kubectl https://dl.k8s.io/release/v1.21.2/bin/linux/amd64/kubectl 
!cp /tmp/kubectl /opt/conda/bin # Move the binary to somewhere on the PATH
!chmod +x /opt/conda/bin/kubectl

## Deploy a Pre-Trained Model

The manifest below defines a `SeldonDeployment` using a pre-trained sklearn model. This leverages Seldon-Core's sklearn server implementation.

In [2]:
%%writefile deployment.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: iris-model
  namespace: seldon
spec:
  name: iris
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1

Writing deployment.yaml


And apply the manifest to the seldon namespace.

In [3]:
!kubectl -n seldon apply -f deployment.yaml

seldondeployment.machinelearning.seldon.io/iris-model created


In [4]:
!kubectl -n seldon rollout status deployment/iris-model-default-0-classifier

deployment "iris-model-default-0-classifier" successfully rolled out


## Call The Model

The model container has downloaded a pre-trained model and instantiated it inside a serving container. You can now call the hosted endpoint.

Seldon-core uses a service mesh to call the endpoint, so here you need to point the call towards the ingress gateway of your service mesh. In this case it's the default Istio ingress gateway and I'm able to use the internal Kubernetes DNS because this notebook is running in the cluster.


In [26]:
import json, urllib

url = "http://istio-ingressgateway.istio-system.svc/seldon/seldon/iris-model/api/v1.0/predictions"
data = { "data": { "ndarray": [[1,2,3,4]] } }
params = json.dumps(data).encode('utf8')
req = urllib.request.Request(url,
                             data=params,
                             headers={'content-type': 'application/json'})
response = urllib.request.urlopen(req)
print(json.dumps(json.loads(response.read()), indent=4, sort_keys=True))

{
    "data": {
        "names": [
            "t:0",
            "t:1",
            "t:2"
        ],
        "ndarray": [
            [
                0.0006985194531162841,
                0.003668039039435755,
                0.9956334415074478
            ]
        ]
    },
    "meta": {
        "requestPath": {
            "classifier": "seldonio/sklearnserver:1.9.0"
        }
    }
}
